In [90]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [91]:
data = pd.read_excel("estimate_power.xlsx")
data.head()

,hour,date_miladi,date_shamsi,power,code,unit_no,fuel_type,mvar,temp,moisture
0,1,2020-01-13,1398/10/23,119,SO,1,A,11.0,3,94.0
1,2,2020-01-13,1398/10/23,119,SO,1,A,11.0,3,96.0
2,3,2020-01-13,1398/10/23,120,SO,1,A,10.0,2,95.0
3,4,2020-01-13,1398/10/23,120,SO,1,A,11.0,2,95.0
4,5,2020-01-13,1398/10/23,121,SO,1,A,11.0,2,95.0


In [92]:
# find categorical variables
categorical = [var for var in data.columns if data[var].dtype=='O']
print('There are {} categorical variables\n'.format(len(categorical)))
print('The categorical variables are :', categorical)

There are 3 categorical variables

The categorical variables are : ['date_shamsi', 'code', 'fuel_type']


In [93]:
# print categorical variables containing missing values
cat1 = [var for var in categorical if data[var].isnull().sum()!=0]
print(data[cat1].isnull().sum())


fuel_type    92375
dtype: int64


In [94]:
data.isnull().sum()

hour               0
date_miladi        0
date_shamsi        0
power              0
code               0
unit_no            0
fuel_type      92375
mvar               0
temp               0
moisture           0
dtype: int64

In [95]:
data['fuel_type'].value_counts()

A    162981
C     11070
B     10701
Name: fuel_type, dtype: int64

In [96]:
# Change object to integer:
data[data["fuel_type"]=="A"] = 1 
data[data["fuel_type"]=="B"] = 2
data[data["fuel_type"]=="C"] = 3 

In [97]:
data['fuel_type'].value_counts()

1    162981
3     11070
2     10701
Name: fuel_type, dtype: int64

In [98]:
data.isnull().sum()

hour               0
date_miladi        0
date_shamsi        0
power              0
code               0
unit_no            0
fuel_type      92375
mvar               0
temp               0
moisture           0
dtype: int64

In [99]:
data["fuel_type"].fillna(method='bfill', inplace=True)

In [100]:
X = data[["moisture","unit_no","fuel_type","mvar","temp"]]
Y = data["power"]


In [101]:
data.isnull().sum()

hour            0
date_miladi     0
date_shamsi     0
power           0
code            0
unit_no         0
fuel_type      72
mvar            0
temp            0
moisture        0
dtype: int64

In [102]:
data.fillna(data.mode, inplace=True)

In [103]:
data.isnull().sum()

hour           0
date_miladi    0
date_shamsi    0
power          0
code           0
unit_no        0
fuel_type      0
mvar           0
temp           0
moisture       0
dtype: int64

In [104]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
X

,moisture,unit_no,fuel_type,mvar,temp
0,-0.512353,-0.748063,-0.336528,-0.141056,-0.615644
1,-0.512353,-0.748063,-0.336528,-0.141056,-0.615644
2,-0.512353,-0.748063,-0.336528,-0.141056,-0.615644
3,-0.512353,-0.748063,-0.336528,-0.141056,-0.615644
4,-0.512353,-0.748063,-0.336528,-0.141056,-0.615644
...,...,...,...,...,...
277122,-0.189675,1.692781,NaN,3.506404,2.193880
277123,-0.143578,1.692781,NaN,3.437584,2.023605
277124,-0.051384,1.692781,NaN,3.231124,1.938468
277125,-0.097481,1.692781,NaN,3.299944,1.853331


In [105]:
pca = PCA()
pca.fit(X)
transformed = pca.transform(seed)
mean = pca.mean_
FPC = pca.components_[0,:]
plt.scatter(width, length)
plt.arrow(mean[0], mean[1], FPC[0], FPC[1], color='red',width=0.01)
#plt.axis('equal')# Keep axes on same scale
plt.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [106]:
data["fuel_type"].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
277122    False
277123    False
277124    False
277125    False
277126    False
Name: fuel_type, Length: 277127, dtype: bool